### Install Flask package using pip
The exclamation mark '!' is used to execute Linux commands. 
If the Flask package is installed, you can ignore the cell below.

In [179]:
#!pip3 install flask

In [180]:
#!pip install --upgrade flask


In [181]:
#!pip install watchdog

In [182]:
import os
import sqlite3
from datetime import datetime
from livereload import Server
from flask import Flask, render_template, request, jsonify

In [183]:
app = Flask(__name__)

In [184]:
# 資料庫路徑
DATABASE_FOLDER = os.path.join(app.root_path, 'instance')


In [185]:
# 確保所有需要的文件夾存在
for folder in [DATABASE_FOLDER]:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [186]:
DATABASE = os.path.join(DATABASE_FOLDER, 'database.db')

In [187]:
# 定義根目錄
@app.route('/')
def index():
    return render_template('index.html')

In [188]:
# 定義分類目錄入口
@app.route('/classify')
def classify():
    return render_template('class.html')

In [ ]:
#定義身體頁面
@app.route('/body')
def bodyPage():
   return render_template('classTemplate.html', title = "BODY")

In [ ]:
#定義心靈頁面
@app.route('/psycho')
def psychoPage():
   return render_template('classTemplate.html', title = "PSYCHO")

In [ ]:
#定義社會頁面
@app.route('/social')
def socialPage():
   return render_template('classTemplate.html', title = "SOCIAL")

In [ ]:
#定義特殊頁面
@app.route('/special')
def specialPage():
   return render_template('classTemplate.html', title = "SPECIAL")

In [ ]:
#定義其他頁面
@app.route('/extra')
def extraPage():
   return render_template('classTemplate.html', title = "EXTRA")

In [190]:
# 查詢姓名列表
@app.route('/fetchNameList', methods=['GET']) 
def fetchNameList():
    try:
        # 使用 with 語句管理資料庫連線
        with sqlite3.connect(DATABASE) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM nameList")  # 假設表格名稱為 'nameList'，字段名稱為 'name'
            names = [row[0] for row in cursor.fetchall()]
        return jsonify(names)

    except Exception as e:
        return jsonify({'error': str(e)}), 500


In [191]:
# 新增姓名到資料庫
@app.route('/addName', methods=['POST'])
def addName():
    try:
        data = request.get_json()
        new_name = data.get('name')
        if not new_name:
            return jsonify({'error': 'No name provided'}), 400
        
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO nameList (name) VALUES (?)", (new_name,))
        conn.commit()
        conn.close()
        return jsonify({'success': True}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.103:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Nov/2024 17:02:31] "GET /classPage HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 17:02:32] "GET /classPage HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 17:02:32] "GET /classPage HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 17:02:32] "GET /classPage HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 17:02:35] "GET /classPage HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 17:02:35] "GET /classPage HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 17:02:53] "GET /classPage HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 17:02:54] "GET /classPage HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 17:02:57] "GET /classify HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2024 17:02:57] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Nov/2024 17:02:57] "GET /static/js/class.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Nov/2024 17:02:57] "GET /classify HTTP/1.1" 200 -
127.0.0.1 - - [27/N